In [1]:
from data_preprocessing.cleaning import clean_audio_folder
from data_preprocessing.train_test_split import train_test_split_files
from data_preprocessing.merging import merge_audios_by_speakers
from data_preprocessing.splitting import split_audio_files
from data_preprocessing.spectrogram_converter import convert_audio_files_to_images

p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [1]:
train_test_split_files(
    '../data/tsrc/train', 
    '../data/tsrc_train_test/train', 
    '../data/tsrc_train_test/test', 
    '../data/tsrc_train_test/validation'
)

clean_audio_folder('../data/tsrc_train_test', '../data/tsrc_cleaned')
merge_audios_by_speakers('../data/tsrc_cleaned', '../data/tsrc_merged')
split_audio_files('../data/tsrc_merged', '../data/tsrc_splitted')

p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Splitting audio files for train data...


100%|██████████| 1503/1503 [00:09<00:00, 164.39it/s]


Splitting audio files for test data...


100%|██████████| 189/189 [00:01<00:00, 182.92it/s]


Splitting audio files for validation data...


100%|██████████| 189/189 [00:01<00:00, 182.90it/s]


In [2]:
convert_audio_files_to_images("../data/tsrc_splitted", "../data/tsrc_sprectrograms")

Converting audio files to images for train data...


 40%|███▉      | 3507/8874 [00:20<00:42, 125.23it/s] p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1941
  warnings.warn(
 47%|████▋     | 4201/8874 [01:23<06:51, 11.34it/s] p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1764
  warnings.warn(
 48%|████▊     | 4231/8874 [01:25<07:07, 10.86it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=530
  warnings.warn(
 51%|█████▏    | 4566/8874 [01:57<07:21,  9.76it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1059
  warnings.warn(
 53%|█████▎    | 4668/8874 [02:06<06:07, 11.43it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: User

Converting audio files to images for test data...


 24%|██▍       | 276/1140 [00:25<01:21, 10.54it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1267
  warnings.warn(
 45%|████▍     | 510/1140 [00:46<01:01, 10.22it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=353
  warnings.warn(
 89%|████████▉ | 1019/1140 [01:33<00:11, 10.58it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1817
  warnings.warn(
 98%|█████████▊| 1117/1140 [01:42<00:02, 10.60it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1591
  warnings.warn(
100%|██████████| 1140/1140 [01:44<00:00, 10.95it/s]


Converting audio files to images for validation data...


  6%|▋         | 71/1115 [00:06<01:39, 10.48it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=882
  warnings.warn(
 22%|██▏       | 246/1115 [00:22<01:20, 10.84it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1588
  warnings.warn(
 67%|██████▋   | 746/1115 [01:08<00:32, 11.46it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=626
  warnings.warn(
 89%|████████▉ | 991/1115 [01:30<00:11, 10.85it/s]p:\MINI8\I2ML\intro-to-ml\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=177
  warnings.warn(
100%|██████████| 1115/1115 [01:42<00:00, 10.90it/s]
